In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None
from sklearn import preprocessing
import numpy as np
from statsmodels.stats import inter_rater as irr
pd.options.display.max_colwidth = 240

In [9]:
juanjo = pd.read_csv('../Data/Data_Annotated/latest_juanjo_annotations.csv', sep = '\t')
juanma = pd.read_csv('../Data/Data_Annotated/latest_juanma_annotations.csv', sep = '\t')
ger = pd.read_csv('../Data/Data_Annotated/latest_ger_annotations.csv', sep = '\t')

In [10]:
juanjo = juanjo.rename(columns = {'sentimento':'sentiment_juanjo', 'relevancia': 'relevancia_juanjo', 'emocion': 'emocion_juanjo', 'ironia': 'ironia_juanjo'})
juanma = juanma.rename(columns = {'sentimento':'sentiment_juanma', 'relevancia': 'relevancia_juanma', 'emocion': 'emocion_juanma', 'ironia': 'ironia_juanma'})
ger = ger.rename(columns = {'sentimento':'sentiment_ger', 'relevancia': 'relevancia_ger', 'emocion': 'emocion_ger', 'ironia': 'ironia_ger'})

In [11]:
merged_df = pd.merge(pd.merge(juanjo[['text', 'tweet_id', 'sentiment_juanjo']],
                     juanma[['tweet_id', 'sentiment_juanma']], on=['tweet_id']),
                    ger[['tweet_id', 'sentiment_ger']], on=['tweet_id'])

In [15]:
merged_df = pd.merge(pd.merge(juanjo[['text', 'tweet_id', 'sentiment_juanjo']],
                     juanma[['tweet_id', 'sentiment_juanma']], on=['tweet_id']),
                    ger[['tweet_id', 'sentiment_ger']], on=['tweet_id'])
merged_relevancia = pd.merge(pd.merge(juanjo[['text', 'tweet_id', 'relevancia_juanjo']],
                             juanma[['tweet_id', 'relevancia_juanma']], on=['tweet_id']),
                            ger[['tweet_id', 'relevancia_ger']], on=['tweet_id'])
merged_emocion = pd.merge(pd.merge(juanjo[['text', 'tweet_id', 'emocion_juanjo']],
                          juanma[['tweet_id', 'emocion_juanma']], on=['tweet_id']),
                         ger[['tweet_id', 'emocion_ger']], on=['tweet_id'])
merged_ironia = pd.merge(pd.merge(juanjo[['text', 'tweet_id', 'ironia_juanjo']],
                         juanma[['tweet_id', 'ironia_juanma']], on=['tweet_id']),
                        ger[['tweet_id', 'ironia_ger']], on=['tweet_id'])

In [16]:
dict_emociones = {
    'alegria' : ['Alegria', 'Confianza', 'Admiración','Agrado'],
    'miedo' : ['Miedo', 'Incertidumbre'],
    'sorpresa' : ['Sorpresa', 'Asombro'],
    'tristeza' : ['Tristeza' , 'Decepción'],
    'asco' : ['Asco', 'Desagrado'],
    'ira' : ['Ira', 'Odio'],
    'otra' : ['Otra'] }

In [17]:
for i  in dict_emociones.keys():
    merged_df[i + '_juanma'] = merged_df['sentiment_juanma'].str.contains('|'.join(dict_emociones[i]))
    merged_df[i + '_juanjo'] = merged_df['sentiment_juanjo'].str.contains('|'.join(dict_emociones[i]))
    merged_df[i + '_ger'] = merged_df['sentiment_ger'].str.contains('|'.join(dict_emociones[i]))

In [18]:
merged_df = merged_df.fillna(False)

In [19]:
merged_df['labels'] = merged_df.apply(lambda x: ', '.join(x.index[x == True]), axis=1)

In [20]:
raw_counts = {}
for i in dict_emociones.keys():
    for j in dict_emociones[i]:
        raw_emotion = j
        raw_counts[j] = len(merged_df[(merged_df['sentiment_juanjo'].str.contains(raw_emotion).fillna(False)) | (merged_df['sentiment_juanma'].str.contains(raw_emotion).fillna(False))]) 

In [21]:
raw_emotion = 'Agrado'
merged_df[(merged_df['sentiment_juanjo'].str.contains(raw_emotion).fillna(False)) | (merged_df['sentiment_juanma'].str.contains(raw_emotion).fillna(False))]

,text,tweet_id,sentiment_juanjo,sentiment_juanma,sentiment_ger,alegria_juanma,alegria_juanjo,alegria_ger,miedo_juanma,miedo_juanjo,...,asco_juanma,asco_juanjo,asco_ger,ira_juanma,ira_juanjo,ira_ger,otra_juanma,otra_juanjo,otra_ger,labels
26,"#MiVotoEsPetroYFrancia, así tendremos autonomía de poderes, ellos respetan las cortes y toda la justicia.",1528483611061719041,Agrado,Confianza,"{""choices"": [""Admiración"", ""Alegria"", ""Agrado""]}",True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,"alegria_juanma, alegria_juanjo, alegria_ger"
40,#MiVotoEsPetroYFrancia y el de la mayoría de mi familia también. El cambio en primera.,1528879175779635200,Agrado,Confianza,Confianza,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,"alegria_juanma, alegria_juanjo, alegria_ger"
51,Gran manifestación en Medellín. #FedericoEsColombia,1528571715667648513,"{""choices"": [""Asombro"", ""Agrado""]}",Agrado,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,"alegria_juanma, alegria_juanjo, sorpresa_juanjo"
135,"#ElDebateDefinitivo Pensé que Fajardo y Fico traían propuestas pero no. Han copiado las de @petrogustavo pero no saben como implementarlas. Petro en cambio explica el como de sus propuestas, les corrige los cálculos, les explica malent...",1529110594477076480,"{""choices"": [""Desagrado"", ""Agrado""]}",Admiración,"{""choices"": [""Asco"", ""Desagrado""]}",True,True,False,False,False,...,False,True,True,False,False,False,False,False,False,"alegria_juanma, alegria_juanjo, asco_juanjo, asco_ger"


In [22]:
raw_counts

{'Alegria': 14,
 'Confianza': 94,
 'Admiración': 40,
 'Agrado': 4,
 'Miedo': 25,
 'Incertidumbre': 10,
 'Sorpresa': 2,
 'Asombro': 7,
 'Tristeza': 2,
 'Decepción': 43,
 'Asco': 46,
 'Desagrado': 102,
 'Ira': 26,
 'Odio': 0,
 'Otra': 4}

In [130]:
raw_emotion = 'Aceptación'
merged_df[(merged_df['sentiment_juanjo'].str.contains(raw_emotion)) | (merged_df['sentiment_juanma'].str.contains(raw_emotion))]

,text,tweet_id,sentiment_juanjo,sentiment_juanma,Confianza_juanma,Confianza_juanjo,miedo_juanma,miedo_juanjo,sorpresa_juanma,sorpresa_juanjo,...,asco_juanjo,ira_juanma,ira_juanjo,expectativa_juanma,expectativa_juanjo,alegria_juanma,alegria_juanjo,otra_juanma,otra_juanjo,labels
62,"Fajardo comienza el debate mostrando como las agresiones y señalamientos entre Fico y Petro generan una afectación a la población colombiana, afectación en la cual él no quiere participar #YoVotoPorFajardo #ElDebateDefinitivo #DebateEl...",1528905779373391872,Aceptación,Admiración,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,"Confianza_juanma, Confianza_juanjo"


In [24]:
algo = 'sorpresa'
merged_df[(merged_df[algo + '_juanma'] == True) & (merged_df[algo + '_juanjo'] == True)]

,text,tweet_id,sentiment_juanjo,sentiment_juanma,alegria_juanma,alegria_juanjo,miedo_juanma,miedo_juanjo,sorpresa_juanma,sorpresa_juanjo,tristeza_juanma,tristeza_juanjo,asco_juanma,asco_juanjo,ira_juanma,ira_juanjo,otra_juanma,otra_juanjo,labels


In [35]:
full_df_agreement[full_df_agreement['emocion'] == 'alegria']

,text,labels,tweet_id,juanma,juanjo,sentiment_juanma,sentiment_juanjo,emocion
0,"#VotoPor #RodolfoHernandez hombre sabio, sagaz y visionario que transformará el país en potencia mundial, del atraso a ejemplo de desarrollo y crecimiento económico #RodolfoPresidente A los haters les digo, #RodolfoPresidente y si van a decir algo con argumentos no inventos","alegria_juanma, alegria_juanjo",1529572795633414144,True,True,Admiración,Confianza,alegria
1,#FedericoEsColombia este 29 de Mayo salgamos todos a votar por la democracia y la libertad de colombia unidos por nuestro país #FedericoImparable #FedericoEsColombia #FicoPresidente2022,"alegria_juanma, alegria_juanjo",1528415123450343432,True,True,Confianza,Confianza,alegria
2,"Por un cambio, por la esperanza de un mejor país, por la vida de líderes jóvenes y líderes sociales, #MiVotoEsPetroYFrancia","alegria_juanma, alegria_juanjo",1528473443271835653,True,True,Confianza,Confianza,alegria
3,Por que Colombia necesita cambio #MiVotoEsPetroYFrancia,"alegria_juanma, alegria_juanjo",1528584113933934593,True,True,Confianza,Confianza,alegria
4,"#ElMejorCandidatoEs el que le apuesta por la paz, la justicia social y un gobierno para todos.","alegria_juanma, alegria_juanjo",1529768426037821443,True,True,Confianza,Confianza,alegria
7,Es claro quien es el presidente y quienes van a quedar rabones ! Petro los peinó tranquilo ! #ElDebateDefinitivo,"alegria_juanma, alegria_juanjo",1529144849441644545,True,True,Confianza,Agrado,alegria
10,"Ahora que #PetroEsPresidente #NuestraPrioridadEs hacerle seguimiento. Apoyarlo con lo bueno, reprocharle lo malo. Marchar cuando sea necesario y así... Será difícil, pero no imposible. Pero con responsabilidad, no con odio.","alegria_juanma, alegria_juanjo",1539264520631242753,True,True,Confianza,Confianza,alegria
11,¿Cuándo un presidente nos había dicho que nos quiere mucho? Y es cierto 🥺. #PetroEsPresidente.,"alegria_juanma, alegria_juanjo",1538916021280681984,True,True,"{""choices"": [""Alegria"", ""Confianza""]}",Alegria,alegria
13,"¡LLEGO LA HORA DE VIVIR SABROSO, CARAJO! #PetroPresidente #VivirSabroso ❤️❤️❤️","alegria_juanma, alegria_juanjo",1538655871353372673,True,True,"{""choices"": [""Alegria"", ""Confianza""]}",Alegria,alegria
14,"La verdad no pensé que viviría para ver el inicio del cambio...gracias por todo colombianos conscientes y pensantes, los amo 😭😭😭 #PetroEsPresidente","alegria_juanma, alegria_juanjo",1538667002583306245,True,True,"{""choices"": [""Alegria"", ""Confianza""]}",Alegria,alegria


In [65]:
pd.options.display.max_colwidth = 300
full_df_disagreement[full_df_disagreement['emocion'] == 'sorpresa']

,text,labels,tweet_id,juanma,juanjo,sentiment_juanma,sentiment_juanjo,emocion
66,@yescuder #PetroEsPresidente | Le recuerdo que para esta segunda vuelta salió a votar las tribus e indígenas más excluidos en nuestro país. Indígenas total en nuestro país registrado: 1'905.617 de personas También le recuerdo Sigue el Hilo 🧵,sorpresa_juanjo,1538918509450764290,False,True,False,Sorpresa,sorpresa
67,Veo con emoción que muchos de mis amigos reportan masiva asistencia a las urnas. Colombia sí está cambiando. El pueblo despertó. ✊🏻 #Elecciones2022 #EleccionesColombia,"alegria_juanma, alegria_juanjo, sorpresa_juanjo",1530971942110609408,False,True,Confianza,"{""choices"": [""Asombro"", ""Alegria""]}",sorpresa
68,La verdad yo si quisiera saber que le hizo @AlvaroUribeVel a @petrogustavo que no lo suelta para nada. Todo el #ElDebateDefinitivo se la paso uribe uribe uribe y nada mas. Ojalá me respondiera,"sorpresa_juanjo, tristeza_juanjo, asco_juanma",1528956465632903168,False,True,Desagrado,"{""choices"": [""Decepción"", ""Sorpresa""]}",sorpresa


In [138]:
full_df_agreement[full_df_agreement['emocion'] == 'alegria']

,text,labels,juanma,juanjo,sentiment_juanma,sentiment_juanjo,emocion


In [ ]:
full_df[full_df['emocion'].str.contains('')]

In [10]:
merged_df.to_csv('../Data/Data_Annotated/merged_annotations.csv', index=False)

In [23]:
merged_relevancia = merged_relevancia.fillna('No')
merged_relevancia['relevancia_juanjo_bool'] = merged_relevancia['relevancia_juanjo'].map({'Si': True, 'No': False})
merged_relevancia['relevancia_juanma_bool'] = merged_relevancia['relevancia_juanma'].map({'Si': True, 'No': False})
merged_relevancia['relevancia_ger_bool'] = merged_relevancia['relevancia_ger'].map({'Si': True, 'No': False})

In [30]:
arr = np.array(merged_relevancia[['relevancia_juanjo_bool', 'relevancia_juanma_bool', 'relevancia_ger_bool']])
agg = irr.aggregate_raters(arr)
irr.fleiss_kappa(agg[0], method='fleiss')

0.7906417112299454

In [31]:
merged_emocion = merged_emocion.fillna('No')
merged_emocion['emocion_juanjo_bool'] = merged_emocion['emocion_juanjo'].map({'Si': True, 'No': False})
merged_emocion['emocion_juanma_bool'] = merged_emocion['emocion_juanma'].map({'Si': True, 'No': False})
merged_emocion['emocion_ger_bool'] = merged_emocion['emocion_ger'].map({'Si': True, 'No': False})

In [32]:
arr_num = np.array(merged_emocion[['emocion_juanjo_bool','emocion_juanma_bool', 'emocion_ger_bool' ]])
agg_num = irr.aggregate_raters(arr_num)
irr.fleiss_kappa(agg_num[0], method='fleiss')

0.49757449757449823

In [33]:
merged_ironia = merged_ironia.fillna('No')
merged_ironia['ironia_juanjo_bool'] = merged_ironia['ironia_juanjo'].map({'Si': True, 'No': False})
merged_ironia['ironia_juanma_bool'] = merged_ironia['ironia_juanma'].map({'Si': True, 'No': False})
merged_ironia['ironia_ger_bool'] = merged_ironia['ironia_ger'].map({'Si': True, 'No': False})

In [34]:
arr_num = np.array(merged_ironia[['ironia_juanjo_bool', 'ironia_juanma_bool', 'ironia_ger_bool']])
agg_num = irr.aggregate_raters(arr_num)
irr.fleiss_kappa(agg_num[0], method='fleiss')

0.27140198511166136

In [56]:
agreements = {}
for i in dict_emociones.keys():
    arr = np.array(merged_df[[i + '_juanjo',i + '_juanma', i + '_ger' ]])
    agg = irr.aggregate_raters(arr)
    agreements[i] = irr.fleiss_kappa(agg[0], method='fleiss')

In [57]:
agreements

{'alegria': 0.7101458385717879,
 'miedo': 0.4488669905998816,
 'sorpresa': -0.012936610608019742,
 'tristeza': 0.3410939691444601,
 'asco': 0.6088536391755399,
 'ira': 0.1456628477905069,
 'otra': -0.006426735218517274}

In [59]:
df_list = []
emotions_count = {}
for i in dict_emociones.keys():
    emotion_df = merged_df[(merged_df[i + '_juanma']== True) | 
                           (merged_df[i + '_juanjo']== True)| 
                           (merged_df[i + '_ger']== True)][['text', 'labels', 'tweet_id' , i + '_juanma', i +'_juanjo', i +'_ger', 'sentiment_juanma', 'sentiment_juanjo' , 'sentiment_ger']]
    emotion_df = emotion_df.rename(columns= {i + '_juanma': 'juanma', i +'_juanjo': 'juanjo',  i +'_ger': 'ger'})
    emotion_df['emocion'] = i
    emotions_count[i] = len(emotion_df)
    df_list.append(emotion_df)
full_df = pd.concat(df_list, ignore_index=True)
#full_df_disagreement = full_df[full_df['juanma'] != full_df['juanjo']]
#full_df_agreement = full_df[full_df['juanma'] == full_df['juanjo']]

In [60]:
total = sum(emotions_count.values())
avgd_agreement = 0
mean_agreement = 0
for i in agreements.keys():
    avgd_agreement+= agreements[i]*(emotions_count[i]/total)
    mean_agreement+= agreements[i]
mean_agreement = mean_agreement/len(agreements)

In [61]:
avgd_agreement

0.5051707025502078

In [62]:
mean_agreement

0.3193228484936627

In [47]:
dict_sentimientos = {
    'positivo' : ['Alegria', 'Confianza', 'Admiración','Agrado'],
    'negativo' : ['Miedo', 'Incertidumbre','Tristeza' , 'Decepción', 'Asco', 'Desagrado','Ira', 'Odio'],
    'otra_sorpresa' : ['Sorpresa', 'Asombro', 'Otra'],
     }

In [48]:
for i  in dict_sentimientos.keys():
    merged_df[i + '_juanma'] = merged_df['sentiment_juanma'].str.contains('|'.join(dict_sentimientos[i]))
    merged_df[i + '_juanjo'] = merged_df['sentiment_juanjo'].str.contains('|'.join(dict_sentimientos[i]))
    merged_df[i + '_ger'] = merged_df['sentiment_ger'].str.contains('|'.join(dict_sentimientos[i]))

In [65]:
agreements = {}
for i in dict_sentimientos.keys():
    arr = np.array(merged_df[[i + '_juanjo',i + '_juanma', i + '_ger' ]])
    agg = irr.aggregate_raters(arr)
    agreements[i] = irr.fleiss_kappa(agg[0], method='fleiss')

In [66]:
agreements

{'positivo': 0.5897863164085473,
 'negativo': 0.5606850095262633,
 'otra_sorpresa': 0.6146653543307087}

In [67]:
df_list = []
emotions_count = {}
for i in dict_sentimientos.keys():
    emotion_df = merged_df[(merged_df[i + '_juanma']== True) | 
                           (merged_df[i + '_juanjo']== True)| 
                           (merged_df[i + '_ger']== True)][['text', 'labels', 'tweet_id' , i + '_juanma', i +'_juanjo', i +'_ger', 'sentiment_juanma', 'sentiment_juanjo' , 'sentiment_ger']]
    emotion_df = emotion_df.rename(columns= {i + '_juanma': 'juanma', i +'_juanjo': 'juanjo',  i +'_ger': 'ger'})
    emotion_df['emocion'] = i
    emotions_count[i] = len(emotion_df)
    df_list.append(emotion_df)
full_df = pd.concat(df_list, ignore_index=True)
#full_df_disagreement = full_df[full_df['juanma'] != full_df['juanjo']]
#full_df_agreement = full_df[full_df['juanma'] == full_df['juanjo']]

In [68]:
total = sum(emotions_count.values())
avgd_agreement = 0
mean_agreement = 0
for i in agreements.keys():
    avgd_agreement+= agreements[i]*(emotions_count[i]/total)
    mean_agreement+= agreements[i]
mean_agreement = mean_agreement/len(agreements)

In [69]:
avgd_agreement

0.5755635380638484

In [70]:
mean_agreement

0.5883788934218398

In [19]:
pd.options.display.max_colwidth = 300
merged_emocion_disagreement = merged_emocion[(merged_emocion['emocion_juanjo'] == 'No') |(merged_emocion['emocion_juanma'] == 'No')]

In [30]:
to_exclude = merged_relevancia_no[['text','tweet_id']].append(merged_emocion_disagreement[['text','tweet_id']], ignore_index=True).drop_duplicates(subset=['tweet_id'])

In [56]:
disagreement_unique = full_df_disagreement.drop_duplicates(subset=['tweet_id'])[['text','tweet_id']]
disagreement_unique = disagreement_unique.reset_index(drop = True)

In [61]:
df_clean = disagreement_unique[disagreement_unique.merge(to_exclude, how='left', on='tweet_id')['text_y'].isna()]

In [77]:
df_final = df_clean[~df_clean['text'].str.contains('https')]
df_final.to_csv('../Data/Data_Annotated/to_annotate.csv', index=False)